In [1]:
import cv2
import mediapipe as mp
import time
import pygame
from scipy.spatial import distance as dist


pygame 2.6.1 (SDL 2.28.4, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import cv2
import mediapipe as mp
import time
import pygame
from scipy.spatial import distance as dist

# Initialize mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Initialize Pygame for the alert sound
pygame.mixer.init()
pygame.mixer.music.load("alert.wav")  # Ensure you have an 'alert.wav' sound file

# Eye landmarks for EAR calculations
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

# Thresholds and counters
EAR_THRESHOLD = 0.2
CLOSED_EYE_DURATION_THRESHOLD = 1  # Seconds

# Initialize variables
blink_count = 0
start_drowsy_time = None
is_drowsy = False

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vertical_2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horizontal = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Start capturing video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Failed to capture image")
        break
    
    # Convert the BGR frame to RGB before processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get the face landmarks
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract eye landmarks
            left_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) for i in LEFT_EYE]
            right_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) for i in RIGHT_EYE]

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            avg_ear = (left_ear + right_ear) / 2.0

            # Blink detection based on EAR
            if avg_ear < EAR_THRESHOLD:
                if start_drowsy_time is None:
                    start_drowsy_time = time.time()
                elif time.time() - start_drowsy_time >= CLOSED_EYE_DURATION_THRESHOLD:
                    # Trigger alert if eyes are closed for a prolonged duration
                    if not is_drowsy:
                        pygame.mixer.music.play(-1)  # Loop the alert sound
                        is_drowsy = True
                    cv2.putText(frame, "DROWSY! Wake up!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
            else:
                if start_drowsy_time is not None:
                    blink_count += 1
                start_drowsy_time = None
                if is_drowsy:
                    pygame.mixer.music.stop()  # Stop the alert sound
                    is_drowsy = False

            # Display blink count on the screen
            cv2.putText(frame, f"Blinks: {blink_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Draw eye landmarks for visualization
            for point in left_eye + right_eye:
                cv2.circle(frame, (int(point[0]), int(point[1])), 2, (0, 255, 0), -1)
    
    # Display the frame
    cv2.imshow('Drowsiness Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.quit()


c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
